# Datorseenede projekt 
#### Projektet i kursen datorseende

In [13]:
%pip cache purge

Files removed: 884 (2914.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip uninstall torchvision -y


Found existing installation: torchvision 0.21.0+cu126
Uninstalling torchvision-0.21.0+cu126:
  Successfully uninstalled torchvision-0.21.0+cu126
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [15]:

%pip install torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.1 MB 1.3 MB/s eta 0:00:05
   ----- ---------------------------------- 0.8/6.1 MB 1.3 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/6.1 MB 1.3 MB/s eta 0:00:04
   -------- ------------------------------- 1.3/6.1 MB 1.3 MB/s eta 0:00:04
   ---------- ----------------------------- 1.6/6.1 MB 1.3 MB/s eta 0:00:04
   ----------- ---------------------------- 1.8/6.1 MB 1.3 MB/s eta 0:00:04
   ------------- -------------------------- 2.1/6.1 MB 1.3 MB/s eta 0:00:04
   --------------- ------------------------ 2.4/6.1 MB 1.3 MB/s eta 0:00:03
   ----------------- ---------------------- 2.6/6.1 MB 1.3 MB/s eta 0:00:03
   ------------------ --------------------- 2.9/6.1 MB 1.3 MB/s eta 0:00:03
   -------------------- -------------------

### Installationer
Installerar paket som behövs för exekveringen av koden.

In [16]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126Note: you may need to restart the kernel to use updated packages.



### Spjälka data 
Spjälker data för att kunna användas i yolo modellen. Data spjälks till test, train och val.

In [7]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.7,0.2,0.1),group_prefix=None)

Copying files: 5302 files [00:24, 212.72 files/s]


### Testa installation
för att kolla att allt är installerat korrekt och att versionerna matchar

In [19]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


## Utför träningen 
Träna en yolov11 model med datan som vi har, med passande parametrar för egen dator att köra.

In [6]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11s.pt")

### Träna modelen på dataset

In [ ]:
# Train model on custom dataset
results = model.train(
    data="config.yaml",
    project="./output",
    device='cuda:0',
    epochs=100,
    batch=-1,  
    imgsz=512,
    workers=12,
    amp=True, 
    name="yolo11_50",
    exist_ok=True,
    patience=10
)

New https://pypi.org/project/ultralytics/8.3.92 available  Update with 'pip install -U ultralytics'


SyntaxError: '[31m[1mbatch_size[0m' is not a valid YOLO argument. Similar arguments are i.e. ['batch=16'].

    Arguments received: ['yolo', '--f=c:\\Users\\rasmu\\AppData\\Roaming\\jupyter\\runtime\\kernel-v34a4737bf41859a8a37d9c0f756f172d106f6ebda.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'detect', 'classify', 'pose', 'obb', 'segment'})
                MODE (required) is one of frozenset({'predict', 'export', 'benchmark', 'val', 'track', 'train'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at im

### Som följande kör vi validering på vår model

In [4]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning E:\Programming\cv-25-project\dataset\val\labels.cache... 14 images, 104 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.28it/s]


                   all        118         22     0.0846      0.686     0.0928     0.0743
                 skier         11         15     0.0891        0.8        0.1     0.0758
           snowboarder          5          7     0.0801      0.571     0.0855     0.0729
Speed: 1.4ms preprocess, 7.4ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs\detect\val
